# Station WMO Metar

First you have to import the meteomatics module and the datetime module

In [1]:
import datetime as dt
import meteomatics.api as api

Input here your username and password from your meteomatics profile

In [2]:
###Credentials:
username = 'python-community'
password = 'Umivipawe179'

Input here a startdate, an enddate and the time interval, all as datetime-objects. The interval tells you, if you get the data in hourly steps, daily steps or every five minutes in between the startdate and the enddate. Note that the enddate has to be in the past.

In [3]:
startdate_station_ts = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)-dt.timedelta(days=2)
enddate_station_ts = startdate_station_ts + dt.timedelta(days=2)
interval_station_ts = dt.timedelta(hours=1)

Choose the parameters you want to get and put them into a list. Check here which parameters are available: https://www.meteomatics.com/en/api/available-parameters/

In [4]:
parameters_station_ts = ['t_2m:C', 'precip_1h:mm']

Input here 'mix-obs' to get observational data.

In [5]:
model_station_ts = 'mix-obs'

Input here the ID of the station(s) you want to get. You can find the ID through "Find Stations". If you have no indications for a certain station-type, just write None.

In [6]:
wmo_stations = ['066810']
metar_stations = ['EDDF']
mch_stations = None #['SCU']

In the following, the request will start. If there is an error in the request as for example a wrong parameter or a date that doesn't exist, you get a message. Note, that this model 'mix-obs' is not available for the 'python-community' trial account.

In [7]:
print("station wmo + metar ids timeseries:")
try:
    df_sd_ids = api.query_station_timeseries(startdate_station_ts, enddate_station_ts, interval_station_ts,
                                             parameters_station_ts, username, password, model=model_station_ts,
                                             wmo_ids=wmo_stations, metar_ids=metar_stations, mch_ids=mch_stations,
                                             on_invalid='fill_with_invalid', request_type="POST", temporal_interpolation='none')
    print(df_sd_ids.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))


station wmo + metar ids timeseries:
2019-04-10 14:22:19| INFO |Calling URL: https://api.meteomatics.com/2019-04-08T00:00:00+00:00--2019-04-10T00:00:00+00:00:PT1H (username = python-community)
Failed, the exception is Request with valid date 2019-04-08 00:00:00 requires historic data access before 2019-04-09 00:00:00, which is not granted with this subscription type (e.g. trial). The valid time period for this account type starts at 2019-04-09 00:00:00. Please contact sales@meteomatics.com and we are happy to provide an extended trial or an upgrade of your account.


As output you get a pandas dataframe with the station-id and the date in the index.

![13_station_wmo_metar_output](api_demo_images/13_station_wmo_metar_output.png)

Now you can work on the data by using pandas commands. Here are some examples how you can access to the different datapoints.

In [ ]:
maximum_temperature = df_sd_ids['t_2m:C'].max()
minimum_temperature = df_sd_ids['t_2m:C'].min()
mean_temperature = df_sd_ids['t_2m:C'].mean()
just_precipitation = df_sd_ids['precip_1h:mm']
first_value_of_precipitation = df_sd_ids.iloc[0]['precip_1h:mm']
just_at_12UTC = df_sd_ids[df_sd_ids.index.get_level_values('validdate').hour == 12]
just_EDDF_station = df_sd_ids[df_sd_ids.index.get_level_values('station_id').str.contains('EDDF')]